In [1]:
DATA_NAME = 'coco-indoor' 
TRANSFORM = 'gabor'
CHANNEL = 'blue'
PARAM_CSV = 'gabor_new.csv' # only use for Gabor

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_gabor_indoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))
param_df = pd.read_csv(os.path.join(ROOT_DIR, "gabor", PARAM_CSV))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters
elif 'gabor' in TRANSFORM:
    GROUPS = param_df['index']

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) < 100:
            X[:, i] = np.full(bootstrap_size, np.nan)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df = cov_df.dropna(how='all').dropna(axis=1, how='all')
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

index,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
index,,,,,,,,,,,,,,,,,,,,,
0,45240.28,1.30,-0.20,0.41,0.10,-0.15,-0.08,-2.40,0.80,-0.43,...,-0.20,0.34,-0.01,0.70,-0.12,-0.39,0.13,0.09,0.06,0.05
1,1.30,16161.59,0.64,-0.36,-0.10,-0.10,-0.01,1.39,-0.19,-0.36,...,-0.00,-0.00,0.02,0.92,-0.32,-0.14,-0.25,0.04,0.04,0.06
2,-0.20,0.64,5922.50,-0.06,0.00,0.07,0.04,0.15,0.44,-0.20,...,0.04,-0.04,0.03,-0.60,0.51,0.00,-0.23,0.00,0.07,-0.01
3,0.41,-0.36,-0.06,2186.22,-0.05,0.02,-0.01,-0.56,-0.20,0.02,...,-0.03,-0.03,0.01,0.36,0.08,0.10,0.05,0.02,-0.02,-0.01
4,0.10,-0.10,0.00,-0.05,896.79,-0.01,0.00,0.02,-0.00,0.19,...,-0.00,0.01,0.00,-0.02,0.11,-0.06,0.06,0.01,0.02,-0.02
5,-0.15,-0.10,0.07,0.02,-0.01,319.77,-0.00,-0.25,0.07,-0.02,...,-0.01,-0.01,0.00,-0.11,0.02,-0.01,0.04,-0.00,0.00,0.00
6,-0.08,-0.01,0.04,-0.01,0.00,-0.00,201.77,0.08,-0.00,0.07,...,0.02,-0.00,-0.01,-0.05,0.00,0.02,-0.02,0.02,-0.01,0.00
7,-2.40,1.39,0.15,-0.56,0.02,-0.25,0.08,62793.30,-0.14,0.46,...,-0.01,0.11,-0.13,0.51,-1.47,-0.46,-0.50,0.26,0.19,-0.14
8,0.80,-0.19,0.44,-0.20,-0.00,0.07,-0.00,-0.14,22700.11,0.27,...,-0.10,0.04,0.01,0.50,0.90,-0.61,0.17,0.12,-0.01,-0.03


In [10]:
corr_matrix = cov_df / np.sqrt(np.outer(np.diag(cov_df), np.diag(cov_df)))
corr_matrix.round(5)

index,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
index,,,,,,,,,,,,,,,,,,,,,
0,1.00000,0.00005,-0.00001,0.00004,0.00002,-0.00004,-0.00003,-0.00005,0.00002,-0.00002,...,-0.00003,0.00009,-0.00000,0.00002,-0.00000,-0.00002,0.00001,0.00001,0.00001,0.00002
1,0.00005,1.00000,0.00007,-0.00006,-0.00003,-0.00005,-0.00001,0.00004,-0.00001,-0.00003,...,-0.00000,-0.00000,0.00002,0.00003,-0.00002,-0.00001,-0.00004,0.00001,0.00002,0.00004
2,-0.00001,0.00007,1.00000,-0.00002,0.00000,0.00005,0.00004,0.00001,0.00004,-0.00003,...,0.00002,-0.00003,0.00003,-0.00004,0.00005,0.00000,-0.00006,0.00000,0.00005,-0.00001
3,0.00004,-0.00006,-0.00002,1.00000,-0.00003,0.00002,-0.00001,-0.00005,-0.00003,0.00000,...,-0.00003,-0.00004,0.00002,0.00004,0.00001,0.00003,0.00002,0.00001,-0.00003,-0.00001
4,0.00002,-0.00003,0.00000,-0.00003,1.00000,-0.00002,0.00001,0.00000,-0.00000,0.00007,...,-0.00000,0.00001,0.00001,-0.00000,0.00003,-0.00003,0.00004,0.00001,0.00004,-0.00004
5,-0.00004,-0.00005,0.00005,0.00002,-0.00002,1.00000,-0.00000,-0.00006,0.00003,-0.00001,...,-0.00002,-0.00003,0.00002,-0.00003,0.00001,-0.00001,0.00004,-0.00000,0.00001,0.00000
6,-0.00003,-0.00001,0.00004,-0.00001,0.00001,-0.00000,1.00000,0.00002,-0.00000,0.00005,...,0.00004,-0.00001,-0.00004,-0.00002,0.00000,0.00002,-0.00003,0.00005,-0.00004,0.00001
7,-0.00005,0.00004,0.00001,-0.00005,0.00000,-0.00006,0.00002,1.00000,-0.00000,0.00002,...,-0.00000,0.00002,-0.00004,0.00001,-0.00005,-0.00002,-0.00004,0.00004,0.00004,-0.00004
8,0.00002,-0.00001,0.00004,-0.00003,-0.00000,0.00003,-0.00000,-0.00000,1.00000,0.00002,...,-0.00002,0.00001,0.00000,0.00002,0.00005,-0.00005,0.00002,0.00003,-0.00000,-0.00002


In [11]:
np.linalg.norm(cov_df - np.diag(np.diag(cov_df)))  # Remove diagonal elements for visualization

13.618972390428903

In [12]:
pca = PCA()
pca.fit(cov_df)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[9.41923342e+07 6.23919427e+07 5.09472363e+07 4.85527953e+07
 4.36794617e+07 3.75000874e+07 1.22590810e+07 7.90588830e+06
 6.67130287e+06 6.20019064e+06 5.59303114e+06 4.98793960e+06
 1.66369614e+06 1.15093858e+06 8.73734236e+05 8.22025740e+05
 7.21290984e+05 6.47762927e+05 2.47047939e+05 1.44095306e+05
 1.24111876e+05 1.13451026e+05 1.02960636e+05 8.60044777e+04
 3.34194246e+04 2.11617699e+04 1.98830006e+04 1.77929507e+04
 1.43918126e+04 1.17529755e+04 5.04142872e+03 3.44168228e+03
 2.94583130e+03 2.51040932e+03 2.43969316e+03 1.78065907e+03
 1.10889411e+03 8.77386825e+02 6.55182050e+02 6.15065261e+02
 5.24788503e+02 7.68074677e-24]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,-0.032219,-0.005750,-0.001989,-0.000739,-0.000299,-0.000111,-0.000066,0.996776,-0.008735,-0.002837,...,-0.000251,-0.000104,-0.000050,-0.033750,-0.006023,-0.002035,-0.000770,-0.000299,-0.000115,-0.000055
1,-0.084343,-0.006676,-0.002236,-0.000826,-0.000336,-0.000116,-0.000077,0.042746,-0.010568,-0.003198,...,-0.000276,-0.000113,-0.000052,-0.097640,-0.006939,-0.002251,-0.000847,-0.000346,-0.000134,-0.000056
2,-0.456199,-0.003759,-0.001240,-0.000445,-0.000185,-0.000066,-0.000041,0.014334,-0.006142,-0.001759,...,-0.000148,-0.000065,-0.000029,0.884804,-0.003928,-0.001242,-0.000480,-0.000181,-0.000074,-0.000035
3,0.847454,-0.009751,-0.003189,-0.001148,-0.000472,-0.000173,-0.000109,0.033841,-0.016096,-0.004591,...,-0.000399,-0.000158,-0.000075,0.400646,-0.010233,-0.003238,-0.001209,-0.000475,-0.000186,-0.000083
4,0.217471,-0.012989,-0.004165,-0.001497,-0.000612,-0.000218,-0.000141,0.035940,-0.021923,-0.005986,...,-0.000514,-0.000213,-0.000101,0.175945,-0.013561,-0.004239,-0.001578,-0.000629,-0.000248,-0.000111
5,0.131002,-0.018628,-0.005802,-0.002090,-0.000860,-0.000307,-0.000190,0.038391,-0.032723,-0.008402,...,-0.000725,-0.000304,-0.000135,0.117991,-0.019478,-0.005887,-0.002197,-0.000869,-0.000340,-0.000150
6,0.016239,-0.037186,-0.007012,-0.002448,-0.000992,-0.000351,-0.000223,0.010140,0.994815,-0.010743,...,-0.000836,-0.000347,-0.000157,0.015836,-0.041103,-0.007206,-0.002540,-0.001001,-0.000394,-0.000176
7,0.009705,-0.095568,-0.007591,-0.002526,-0.001047,-0.000365,-0.000231,0.006402,0.044133,-0.012227,...,-0.000874,-0.000349,-0.000166,0.009576,-0.134203,-0.007721,-0.002650,-0.001039,-0.000404,-0.000185
8,0.006606,-0.339696,-0.006388,-0.002088,-0.000842,-0.000300,-0.000191,0.004459,0.024121,-0.010669,...,-0.000714,-0.000290,-0.000140,0.006475,0.927949,-0.006587,-0.002190,-0.000867,-0.000335,-0.000151
9,0.009648,0.889015,-0.010297,-0.003388,-0.001365,-0.000492,-0.000306,0.006520,0.033395,-0.017554,...,-0.001143,-0.000474,-0.000217,0.009478,0.267806,-0.010630,-0.003539,-0.001379,-0.000534,-0.000236


In [13]:
cos_dist = spatial.distance.cdist(eigenvectors, np.eye(cov_df.shape[0]), metric='cosine')
cos_dist =  [np.min(row) for row in cos_dist]
cos_dist


[0.003223880399980117,
 0.011936915094298062,
 0.11519642711284428,
 0.15254555471439735,
 0.07969959850522579,
 0.04207438606419944,
 0.005185166241625905,
 0.018633925552886232,
 0.07205129013415523,
 0.11098456985106364,
 0.13954567640449933,
 0.12637702182254407,
 0.008187480282405568,
 0.016674737967657838,
 0.1418429596289238,
 0.18313014977908693,
 0.29117137675061944,
 0.345262787991643,
 0.009985889742271548,
 0.03624794736463821,
 0.09084277277174269,
 0.18095615324615122,
 0.2560669078554453,
 0.17872660846437016,
 0.019134504566308164,
 0.09639741712398464,
 0.26496925163174845,
 0.3511742259301497,
 0.22109142927997383,
 0.26163791316537155,
 0.03765064146887376,
 0.0838833561388348,
 0.1306994950123942,
 0.24592552666116274,
 0.2015231391081196,
 0.5145440551941873,
 0.17162412285841844,
 0.24198125838923745,
 0.1871946688313667,
 0.2719759017344404,
 0.12641608671814297,
 0.5714702472456937]